In [1]:
import pandas as pd
import pybaseball
from pybaseball import statcast
pd.set_option('display.max_columns', 500)

In [2]:
def fetch_statcast_data(start_date, end_date):
    # Fetch data from Statcast
    data = statcast(start_date, end_date)
    return data

In [3]:
def process_data(data):
    # Filter necessary columns
    data = data[['game_date', 'batter', 'pitcher', 'pitch_type', 'events', 'at_bat_number','zone', 'outs_when_up','p_throws','stand','type','bb_type',
                 'pitch_number']]
    
    # Drop rows where pitch_type is NaN (happens occasionally in real datasets)
    data = data.dropna(subset=['pitch_type'])

    # Group data by game, batter, pitcher, and at_bat_number, then sort by pitch_number
    # Use apply to ensure the group maintains DataFrame format after sorting
    grouped = data.groupby(['game_date', 'batter', 'pitcher', 'pitch_type', 'events', 'at_bat_number','zone', 'outs_when_up','p_throws','stand','type','bb_type']).apply(lambda x: x.sort_values('pitch_number')).reset_index(drop=True)

    # We can't directly iterate with multiple unpacks if using apply without specifying 'group_keys=False' in the groupby
    # However, by resetting the index, we revert to a single DataFrame and thus lose the direct tuple unpacking ability
    # Create a DataFrame to store the sequence, pitcher ID, and at-bat outcome
    at_bats = []
    for _, group in grouped.groupby(['game_date', 'batter', 'pitcher', 'at_bat_number']):
        sequence = ','.join(group['pitch_type'].tolist())
        outcome = group['events'].iloc[-1] if not pd.isna(group['events'].iloc[-1]) else 'No event'
        pitcher_id = group['pitcher'].iloc[0]
        at_bats.append([sequence, pitcher_id, outcome])

    # Convert list to DataFrame
    result_df = pd.DataFrame(at_bats, columns=['Pitch Sequence', 'Pitcher ID', 'At-Bat Outcome'])
    return result_df


In [10]:
def consolidate_pitch_types_cur(data):
    # Mapping dictionary for pitch types
    pitch_type_map = {
        'CH': 'CH',  # Changeup
        'CU': 'CB',  # Curveball
        'FC': 'FC',  # Cutter
        'EP': 'CH',  # Eephus
        'FO': 'CH',  # Forkball
        'FF': 'FF',  # Four-Seam Fastball
        'KN': 'KN',  # Knuckleball
        'KC': 'CB',  # Knuckle-curve
        'SC': 'CH',  # Screwball
        'SI': 'SI',  # Sinker
        'SL': 'SL',  # Slider
        'SV': 'CB',  # Slurve
        'FS': 'FS',  # Splitter
        'ST': 'ST',  # Sweeper
        'FA': 'FF',  # Fastball
        'PO': 'PO',  # Pitch out
        'CS': 'CB'   # Slow Curveball
    }
    
    # Apply the mapping to the pitch_type column
    df1 = data.copy()
    df1['pitch_type'] = df1['pitch_type'].map(pitch_type_map).fillna(data['pitch_type'])
    return df1

In [15]:
def consolidate_pitch_types_new(data):
    # Mapping dictionary for pitch types
    pitch_type_map = {
        'CH': 'CH',  # Changeup
        'CU': 'CB',  # Curveball
        'FC': 'FC',  # Cutter
        'EP': 'CH',  # Eephus
        'FO': 'CH',  # Forkball
        'FF': 'FF',  # Four-Seam Fastball
        'KN': 'KN',  # Knuckleball
        'KC': 'CB',  # Knuckle-curve
        'SC': 'CH',  # Screwball
        'SI': 'SI',  # Sinker
        'SL': 'SL',  # Slider
        'SV': 'CB',  # Slurve
        'FS': 'CH',  # Splitter
        'ST': 'SL',  # Sweeper
        'FA': 'FF',  # Fastball
        'PO': 'PO',  # Pitch out
        'CS': 'CB'   # Slow Curveball
    }
    
    # Apply the mapping to the pitch_type column
    df2 = data.copy()
    df2['pitch_type'] = df2['pitch_type'].map(pitch_type_map).fillna(data['pitch_type'])
    return df2

In [6]:
#pybaseball.cache.enable()
pybaseball.cache.disable()

In [7]:
start_date = '2017-04-01'
end_date = '2023-10-30'
data = fetch_statcast_data(start_date, end_date)

This is a large query, it may take a moment to complete


C:\Users\sid22\anaconda3\envs\nlu\lib\site-packages\pybaseball\statcast.py:50: UserWarning: 
That's a nice request you got there. It'd be a shame if something were to happen to it.
We strongly recommend that you enable caching before running this. It's as simple as `pybaseball.cache.enable()`.
Since the Statcast requests can take a *really* long time to run, if something were to happen, like: a disconnect;
gremlins; computer repair by associates of Rudy Giuliani; electromagnetic interference from metal trash cans; etc.;
you could lose a lot of progress. Enabling caching will allow you to immediately recover all the successful
subqueries if that happens.
  warnings.warn(_OVERSIZE_WARNING)


Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates
Skipping offseason dates


  0%|                                                                                         | 0/1472 [00:00<?, ?it/s]C:\Users\sid22\anaconda3\envs\nlu\lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  0%|                                                                               | 1/1472 [00:03<1:23:38,  3.41s/it]C:\Users\sid22\anaconda3\envs\nlu\lib\site-packages\pybaseball\datahelpers\postprocessing.py:59: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_datetime without passing `errors` and catch exceptions explicitly instead
  data_copy[column] = data_copy[column].apply(pd.to_datetime, errors='ignore', format=date_format)
  0%|                                         

In [8]:
data.sort_values('game_date', inplace=True)

In [9]:
data.columns

Index(['pitch_type', 'game_date', 'release_speed', 'release_pos_x',
       'release_pos_z', 'player_name', 'batter', 'pitcher', 'events',
       'description', 'spin_dir', 'spin_rate_deprecated',
       'break_angle_deprecated', 'break_length_deprecated', 'zone', 'des',
       'game_type', 'stand', 'p_throws', 'home_team', 'away_team', 'type',
       'hit_location', 'bb_type', 'balls', 'strikes', 'game_year', 'pfx_x',
       'pfx_z', 'plate_x', 'plate_z', 'on_3b', 'on_2b', 'on_1b',
       'outs_when_up', 'inning', 'inning_topbot', 'hc_x', 'hc_y',
       'tfs_deprecated', 'tfs_zulu_deprecated', 'fielder_2', 'umpire', 'sv_id',
       'vx0', 'vy0', 'vz0', 'ax', 'ay', 'az', 'sz_top', 'sz_bot',
       'hit_distance_sc', 'launch_speed', 'launch_angle', 'effective_speed',
       'release_spin_rate', 'release_extension', 'game_pk', 'pitcher.1',
       'fielder_2.1', 'fielder_3', 'fielder_4', 'fielder_5', 'fielder_6',
       'fielder_7', 'fielder_8', 'fielder_9', 'release_pos_y',
       'estima

In [12]:
data_pt_cur = consolidate_pitch_types_cur(data)

In [17]:
data_pt_new = consolidate_pitch_types_new(data)

In [16]:
data_pt_cur = data_pt_cur.iloc[2:]

In [17]:
data_pt_cur[['game_date', 'pitch_number','batter', 'pitcher', 'pitch_type', 'events', 'at_bat_number','zone', 'outs_when_up','p_throws','stand','type','bb_type','balls','strikes', 'player_name','on_3b', 'on_2b', 'on_1b']].head(10)

,game_date,pitch_number,batter,pitcher,pitch_type,events,at_bat_number,zone,outs_when_up,p_throws,stand,type,bb_type,balls,strikes,player_name,on_3b,on_2b,on_1b
786,2017-04-02,1,592178,593372,SI,NaN,2,9,0,R,R,S,NaN,0,0,"Martinez, Carlos",<NA>,<NA>,656941
760,2017-04-02,2,592178,593372,SI,NaN,2,6,0,R,R,B,NaN,0,1,"Martinez, Carlos",<NA>,<NA>,656941
737,2017-04-02,3,592178,593372,SL,NaN,2,14,0,R,R,B,NaN,1,1,"Martinez, Carlos",<NA>,<NA>,656941
727,2017-04-02,4,592178,593372,SL,NaN,2,8,0,R,R,S,NaN,2,1,"Martinez, Carlos",<NA>,<NA>,656941
687,2017-04-02,5,592178,593372,FF,strikeout,2,1,0,R,R,S,NaN,2,2,"Martinez, Carlos",<NA>,<NA>,656941
663,2017-04-02,1,519203,593372,FF,NaN,3,11,1,R,L,B,NaN,0,0,"Martinez, Carlos",<NA>,<NA>,656941
647,2017-04-02,2,519203,593372,CH,NaN,3,7,1,R,L,S,NaN,1,0,"Martinez, Carlos",<NA>,<NA>,656941
639,2017-04-02,3,519203,593372,FF,single,3,6,1,R,L,X,line_drive,1,1,"Martinez, Carlos",<NA>,<NA>,656941
614,2017-04-02,1,450314,593372,CH,NaN,4,14,1,R,L,B,NaN,0,0,"Martinez, Carlos",<NA>,656941,519203
594,2017-04-02,2,450314,593372,CH,NaN,4,14,1,R,L,S,NaN,1,0,"Martinez, Carlos",<NA>,656941,519203


In [18]:
data_pt_new.pitch_type.value_counts(normalize=True)

pitch_type
FF    0.346329
SL    0.186511
SI    0.169469
CH    0.125275
CB    0.104744
FC    0.066763
KN    0.000822
PO    0.000086
Name: proportion, dtype: float64

In [19]:
data_pt_cur.pitch_type.value_counts(normalize=True)

pitch_type
FF    0.346329
SI    0.169469
SL    0.166628
CH    0.108219
CB    0.104744
FC    0.066763
ST    0.019883
FS    0.017057
KN    0.000822
PO    0.000086
Name: proportion, dtype: float64

In [20]:
data_toexport_cur = data_pt_cur[['game_date', 'pitch_number','batter', 'pitcher', 'pitch_type', 'events', 'at_bat_number','zone', 'outs_when_up','p_throws','stand','type','bb_type','balls','strikes', 'player_name','on_3b', 'on_2b', 'on_1b']]

In [21]:
data_toexport_new = data_pt_new[['game_date', 'pitch_number','batter', 'pitcher', 'pitch_type', 'events', 'at_bat_number','zone', 'outs_when_up','p_throws','stand','type','bb_type','balls','strikes', 'player_name','on_3b', 'on_2b', 'on_1b']]

In [22]:
data_toexport_cur.to_csv('../../data/raw_data_curmap.csv', index=False)

In [23]:
data_toexport_new.to_csv('../../data/raw_data_newmap.csv', index=False)

In [24]:
data_toexport_new.head()

,game_date,pitch_number,batter,pitcher,pitch_type,events,at_bat_number,zone,outs_when_up,p_throws,stand,type,bb_type,balls,strikes,player_name,on_3b,on_2b,on_1b
901,2017-04-02,1,656941,593372,FF,NaN,1,12,0,R,L,B,NaN,0,0,"Martinez, Carlos",<NA>,<NA>,<NA>
772,2017-04-02,4,592450,643493,FF,fielders_choice,75,5,0,R,R,X,ground_ball,2,1,"Pruitt, Austin",<NA>,<NA>,452104
744,2017-04-02,1,543305,643493,CB,NaN,76,9,0,R,L,S,NaN,0,0,"Pruitt, Austin",<NA>,452104,592450
725,2017-04-02,2,543305,643493,CH,NaN,76,14,0,R,L,B,NaN,0,1,"Pruitt, Austin",<NA>,452104,592450
706,2017-04-02,3,543305,643493,SI,NaN,76,13,0,R,L,B,NaN,1,1,"Pruitt, Austin",<NA>,452104,592450


In [43]:
data_toexport.head()

,game_date,pitch_number,batter,pitcher,balls,strikes,type,pitch_type,events,at_bat_number,player_name
743,2017-04-02,1,543305,643493,0,0,S,CB,NaN,76,"Pruitt, Austin"
715,2017-04-02,2,543305,643493,0,1,B,CH,NaN,76,"Pruitt, Austin"
692,2017-04-02,3,543305,643493,1,1,B,SI,NaN,76,"Pruitt, Austin"
671,2017-04-02,4,543305,643493,2,1,X,SI,single,76,"Pruitt, Austin"
901,2017-04-02,1,474892,517008,0,0,S,FC,NaN,77,"Colomé, Alex"


In [ ]:
result_df = process_data(data_pt_cur)
print(result_df.head())

In [11]:
data_pt[data_pt['pitcher']==621107]['player_name']

83      Eflin, Zach
92      Eflin, Zach
96      Eflin, Zach
101     Eflin, Zach
107     Eflin, Zach
           ...     
3860    Eflin, Zach
3993    Eflin, Zach
4188    Eflin, Zach
4258    Eflin, Zach
4344    Eflin, Zach
Name: player_name, Length: 2652, dtype: object

In [12]:
result_df.shape

(185538, 3)

In [13]:
result_df.head()

,Pitch Sequence,Pitcher ID,At-Bat Outcome
0,SI,621107,field_error
1,"SI,CB,FC,SI,CB,SI,FF",621107,single
2,"ST,ST,SI,SI,ST,ST",676534,walk
3,"SI,ST,SI,ST,SI",687330,grounded_into_double_play
4,"FF,FF,FF,SL,FF,SL",477132,strikeout


In [14]:
result_df.to_csv('../../data/sequence_data_opt.csv', index=False)